# Personality Atlas — Quick Start

Personality psychology has a fragmentation problem. Clinical psychologists use the MMPI and SCID. Trait researchers use OCEAN and HEXACO. Organizational consultants reach for DISC or MBTI. Each tradition has its own instruments, its own factor structures, and its own vocabulary — and almost no shared infrastructure for comparing constructs across traditions.

This notebook walks through a **Computational Atlas** that builds that infrastructure. It embeds 44 personality models — 6,694 traits across 358 factors — into a single vector space where any trait from any tradition can be searched, compared, and classified.

**What you will see:**
- Load any of the 44 models with the same 10 lines of code (Section 2)
- Visualize how 7 research traditions organize in embedding space (Section 4)
- Query a single trait and find related constructs across all 44 models (Section 5)
- Test whether classifiers generalize to novel items they have never seen (Section 7)
- Compare 1536 vs 3072-dim embeddings from Hugging Face (Section 8)
- Route 222 DSM-5 disorders through the atlas as a clinical stress test (Section 9)

Everything runs from pre-computed assets in the repository. No API keys, no database, no waiting.

> Raetano, J., Gregor, J., & Tamang, S. (2026). *A Survey and Computational Atlas of Personality Models.* ACM TIST. Under review.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Wildertrek/survey/blob/main/notebooks/atlas_quick_start.ipynb)

## 1. Setup

Clone the atlas repository and install FAISS (the only dependency Colab does not already have). Takes about 30 seconds.

In [ ]:
# Clone the atlas repository (skip if already cloned)
import os
if not os.path.exists("atlas"):
    !git clone --depth 1 https://github.com/Wildertrek/survey.git atlas
else:
    print("Atlas already cloned — skipping.")

In [ ]:
# Install dependencies (uses Colab's pre-installed sklearn — no version conflicts)
!pip install -q faiss-cpu

# Suppress sklearn version warning (models trained with 1.5.0, works fine with newer versions)
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")

## 2. Load and Predict — Any of the 44 Models

Every model in the atlas follows the same file convention: a **dataset CSV** (traits labeled by factor), a **pre-computed embedding CSV** (1,536-dim vectors from OpenAI `text-embedding-3-small`), and a **trained Random Forest** classifier with its label encoder.

Change `MODEL` to any model abbreviation — `ocean`, `hex`, `mmpi`, `scid`, `npi`, `dt4`, `tci`, `stbv`, etc. — and the same code loads, classifies, and reports. That uniformity is the point: one interface for 44 models from seven different research traditions.

In [ ]:
import ast
import numpy as np
import pandas as pd
import joblib

# Pick any model abbreviation: ocean, hex, mbti, mmpi, scid, npi, dt4, tci, ...
MODEL = "ocean"

df = pd.read_csv(f"atlas/datasets/{MODEL}.csv")
embeddings = pd.read_csv(f"atlas/Embeddings/{MODEL}_embeddings.csv")
model = joblib.load(f"atlas/models/{MODEL}_rf_model.pkl")
encoder = joblib.load(f"atlas/models/{MODEL}_label_encoder.pkl")

X = np.array([ast.literal_eval(e) for e in embeddings["Embedding"]])
predictions = encoder.inverse_transform(model.predict(X))
accuracy = (predictions == df["Factor"].values).mean()

print(f"{MODEL.upper()}: {len(df)} traits, {len(set(predictions))} factors, accuracy = {accuracy:.1%}")
print(f"Factors: {sorted(set(predictions))}")

## 3. Atlas Overview — All 44 Models

How well does each classifier separate its own model's factors? The loop below runs every model and reports accuracy on the full dataset.

Most models score above 95%. With 1,536 embedding dimensions and only a handful of factors, a Random Forest has plenty of room to find separating boundaries. High accuracy here is a necessary condition — it proves the lexical encoding captures meaningful distinctions between factors — but it is not a generalization test. We address that in Section 7 with items the classifiers have never seen.

In [ ]:
import os

slugs = sorted([f.replace(".csv", "") for f in os.listdir("atlas/datasets") if f.endswith(".csv")])
print(f"Atlas contains {len(slugs)} personality models:\n")

results = []
for slug in slugs:
    df = pd.read_csv(f"atlas/datasets/{slug}.csv")
    n_factors = df["Factor"].nunique()
    
    model = joblib.load(f"atlas/models/{slug}_rf_model.pkl")
    enc = joblib.load(f"atlas/models/{slug}_label_encoder.pkl")
    emb = pd.read_csv(f"atlas/Embeddings/{slug}_embeddings.csv")
    X = np.array([ast.literal_eval(e) for e in emb["Embedding"]])
    preds = enc.inverse_transform(model.predict(X))
    acc = (preds == df["Factor"].values).mean()
    
    results.append({"Model": slug.upper(), "Traits": len(df), "Factors": n_factors, "Accuracy": f"{acc:.1%}"})

results_df = pd.DataFrame(results).sort_values("Accuracy", ascending=False)
results_df.index = range(1, len(results_df) + 1)
results_df

## 4. Cross-Model PCA — How 6,694 Traits Organize in Embedding Space

Stack every trait from all 44 models into one matrix and run PCA. Two patterns emerge:

**Theoretical neighbors become geometric neighbors.** Clinical models (MMPI, SCID, BDI) cluster together. Narcissism instruments (NPI, PNI, Dark Triad) form their own region. Trait models (OCEAN, HEXACO) land nearby. The embedding space recovers relationships that were never explicitly encoded — they fall out of the geometry.

**The space is genuinely high-dimensional.** 50 principal components capture a modest share of total variance, meaning personality traits spread across many independent semantic directions. That is consistent with constructs that evolved independently across seven research traditions over the past century.

The three figures below reproduce the paper's PCA analysis (Figures 5-8).

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Category assignments for the 7-category taxonomy
CATEGORIES = {
    "Trait-Based": ["ocean", "hex", "mbti", "epm", "sixteenpf", "ftm"],
    "Narcissism-Based": ["npi", "pni", "hsns", "dtm", "dt4", "ffni", "ffni_sf", "narq", "mcmin", "ipn"],
    "Motivational/Value": ["stbv", "sdt", "rft", "aam", "mst", "cs"],
    "Cognitive/Learning": ["pct", "cest", "scm", "fsls"],
    "Clinical/Health": ["mmpi", "scid", "bdi", "gad7", "wais", "tci", "mcmi", "tmp", "rit", "tat"],
    "Interpersonal/Conflict": ["disc", "tki"],
    "Application-Specific": ["riasec", "cmoa", "tei", "bt", "em", "papc"]
}
slug_to_cat = {s: c for c, slugs in CATEGORIES.items() for s in slugs}

# Load all embeddings
all_vecs, all_labels, all_cats = [], [], []
for slug in slugs:
    df = pd.read_csv(f"atlas/datasets/{slug}.csv")
    emb = pd.read_csv(f"atlas/Embeddings/{slug}_embeddings.csv")
    X = np.array([ast.literal_eval(e) for e in emb["Embedding"]])
    all_vecs.append(X)
    all_labels.extend([slug.upper()] * len(X))
    all_cats.extend([slug_to_cat.get(slug, "Unknown")] * len(X))

X_all = np.vstack(all_vecs)
n_unknown = sum(1 for c in all_cats if c == "Unknown")
print(f"Loaded {X_all.shape[0]} embeddings ({X_all.shape[1]}-dim) from {len(slugs)} models")
if n_unknown > 0:
    unknown_models = sorted(set(lbl for lbl, cat in zip(all_labels, all_cats) if cat == "Unknown"))
    print(f"WARNING: {n_unknown} embeddings from {len(unknown_models)} models have Unknown category: {unknown_models}")

In [ ]:
# PCA — Scree plot (Paper Figure 5)
pca = PCA(n_components=50)
X_pca = pca.fit_transform(X_all)
cumvar = np.cumsum(pca.explained_variance_ratio_) * 100

fig, ax = plt.subplots(figsize=(10, 5))
ax.bar(range(1, 51), pca.explained_variance_ratio_ * 100, alpha=0.6, label="Individual")
ax.plot(range(1, 51), cumvar, "r-o", markersize=3, label="Cumulative")
ax.axhline(y=cumvar[-1], color="gray", linestyle="--", alpha=0.5)
ax.set_xlabel("Principal Component")
ax.set_ylabel("Variance Explained (%)")
ax.set_title(f"PCA Scree Plot — {X_all.shape[0]} Trait Embeddings from 44 Models\n50 PCs capture {cumvar[-1]:.1f}% of variance")
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
# PCA — Model centroids in PC1-PC2 space (Paper Figure 6)
# Colors match the paper's PCA figures (01_cross_model_pca_analysis.py)
cat_colors = {
    "Trait-Based": "#1f77b4", "Narcissism-Based": "#9467bd",
    "Motivational/Value": "#2ca02c", "Cognitive/Learning": "#ff7f0e",
    "Clinical/Health": "#d62728", "Interpersonal/Conflict": "#e377c2",
    "Application-Specific": "#7f7f7f"
}

centroids = pd.DataFrame({
    "PC1": X_pca[:, 0], "PC2": X_pca[:, 1],
    "Model": all_labels, "Category": all_cats
}).groupby(["Model", "Category"])[["PC1", "PC2"]].mean().reset_index()

fig, ax = plt.subplots(figsize=(12, 8))
for cat, color in cat_colors.items():
    subset = centroids[centroids["Category"] == cat]
    ax.scatter(subset["PC1"], subset["PC2"], c=color, s=80, label=cat, alpha=0.8, edgecolors="white", linewidth=0.5)
    for _, row in subset.iterrows():
        ax.annotate(row["Model"], (row["PC1"], row["PC2"]), fontsize=7, alpha=0.7,
                    xytext=(4, 4), textcoords="offset points")

ax.set_xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% variance)")
ax.set_ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% variance)")
ax.set_title("44 Personality Models in PC1-PC2 Space (Model Centroids)")
ax.legend(loc="best", fontsize=8)
plt.tight_layout()
plt.show()

In [ ]:
# PCA — All traits colored by category (Paper Figure 8)
fig, ax = plt.subplots(figsize=(12, 8))
for cat, color in cat_colors.items():
    mask = [c == cat for c in all_cats]
    ax.scatter(X_pca[mask, 0], X_pca[mask, 1], c=color, s=4, alpha=0.3, label=cat)

ax.set_xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% variance)")
ax.set_ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% variance)")
ax.set_title(f"All {X_all.shape[0]} Trait Embeddings — 44 Models, 7 Categories")
ax.legend(markerscale=5, fontsize=8)
plt.tight_layout()
plt.show()

## 5. Cross-Model Search with FAISS

This is what the atlas is for. With every trait from every model in a single FAISS index, you can take any personality construct and retrieve its nearest neighbors *across all 44 models and all 7 categories*.

A clinician studying depression can query a BDI item and find related constructs in the MMPI, TCI, and motivational models. A researcher working on narcissism can trace how Machiavellianism connects to MMPI Psychopathic Deviate and OCEAN low Agreeableness. These cross-tradition connections would normally require deep expertise spanning multiple literatures. The atlas finds them in milliseconds.

Below we build the index (cosine similarity via inner product on L2-normalized vectors) and run two example queries.

In [ ]:
import faiss

# Build FAISS index over entire atlas
X_norm = X_all / np.linalg.norm(X_all, axis=1, keepdims=True)
index = faiss.IndexFlatIP(X_norm.shape[1])
index.add(X_norm.astype(np.float32))

# Build metadata for lookups
all_factors = []
for slug in slugs:
    df = pd.read_csv(f"atlas/datasets/{slug}.csv")
    all_factors.extend(df["Factor"].values)

print(f"FAISS index: {index.ntotal} vectors, {X_norm.shape[1]}-dim")
print(f"Ready for cross-model personality search")

In [ ]:
# Query: find similar traits across all 44 models
# This demonstrates cross-category retrieval — the atlas's core value
query_slug = "ocean"
query_factor = "Extraversion"

query_df = pd.read_csv(f"atlas/datasets/{query_slug}.csv")
query_emb = pd.read_csv(f"atlas/Embeddings/{query_slug}_embeddings.csv")

# Find first row matching the target factor
idx = query_df[query_df["Factor"] == query_factor].index[0]
q = np.array([ast.literal_eval(query_emb["Embedding"].iloc[idx])]).astype(np.float32)
q = q / np.linalg.norm(q)

D, I = index.search(q, 20)

query_trait = query_df.iloc[idx]
print(f"Query: {query_slug.upper()} / {query_factor} — \"{query_trait['Adjective']}\"\n")
print(f"{'Rank':<5} {'Model':<12} {'Factor':<35} {'Category':<22} {'Score':.5}")
print("-" * 85)
for rank, (i, score) in enumerate(zip(I[0], D[0]), 1):
    print(f"{rank:<5} {all_labels[i]:<12} {all_factors[i]:<35} {all_cats[i]:<22} {score:.4f}")

In [ ]:
# Second query: Dark Triad Machiavellianism → cross-category retrieval
# Shows how a narcissism-based construct connects to clinical, trait, and motivational models
query_slug = "dtm"
query_factor = "Machiavellianism"

query_df = pd.read_csv(f"atlas/datasets/{query_slug}.csv")
query_emb = pd.read_csv(f"atlas/Embeddings/{query_slug}_embeddings.csv")

idx = query_df[query_df["Factor"] == query_factor].index[0]
q = np.array([ast.literal_eval(query_emb["Embedding"].iloc[idx])]).astype(np.float32)
q = q / np.linalg.norm(q)

D, I = index.search(q, 20)

query_trait = query_df.iloc[idx]
print(f"Query: {query_slug.upper()} / {query_factor} — \"{query_trait['Adjective']}\"\n")
print(f"{'Rank':<5} {'Model':<12} {'Factor':<35} {'Category':<22} {'Score':.5}")
print("-" * 85)
for rank, (i, score) in enumerate(zip(I[0], D[0]), 1):
    print(f"{rank:<5} {all_labels[i]:<12} {all_factors[i]:<35} {all_cats[i]:<22} {score:.4f}")

# Count unique categories and models in results
result_cats = set(all_cats[i] for i in I[0])
result_models = set(all_labels[i] for i in I[0])
print(f"\n→ {len(result_cats)} categories, {len(result_models)} models in top 20 — cross-tradition retrieval")

## 6. Lexical Schema — What's Inside Each Model

Each model's dataset is a table of traits. Every trait has five columns: **Adjective** (the root descriptor), **Synonym**, **Verb**, **Noun**, and **Factor** (the classification label). This five-column lexical encoding gives each trait a rich semantic surface for the embedding model to work with — richer than single-word labels, but grounded in actual psychometric vocabulary.

Change `INSPECT` below to browse any model's factor structure and sample adjectives.

In [ ]:
# Inspect any model's lexical schema
INSPECT = "hex"  # change to any slug (ocean, hex, mbti, mmpi, scid, npi, dt4, tci, ...)

df = pd.read_csv(f"atlas/datasets/{INSPECT}.csv")
print(f"{INSPECT.upper()}: {len(df)} traits across {df['Factor'].nunique()} factors\n")

for factor, group in df.groupby("Factor"):
    unique_adj = group["Adjective"].unique()[:5]
    print(f"  {factor} ({len(group)} traits): {', '.join(unique_adj)}, ...")

## 7. Experiment 1 — Novel Item Evaluation

The accuracy numbers in Section 3 measure how well each classifier separates its own training lexicon. That is a reproduction check, not a generalization test. Models with few factors relative to 1,536 embedding dimensions can reach 100% by memorizing the training set.

The real question: **can these classifiers recognize personality items they have never seen?**

We test on **5,052 novel items** generated independently by GPT-4o from factor definitions alone, without access to the training data. Pre-computed 1,536-dim embeddings ship with the repository, so this runs without an API key. The mean accuracy across all 44 models is the generalization number reported in the paper.

In [ ]:
import json
from sklearn.metrics import accuracy_score

# Load pre-computed test items (generated by GPT-4o, embedded offline)
test_items = json.load(open("atlas/data/test_items/test_items.json"))
test_emb = np.load("atlas/data/test_items/test_items_embeddings.npz")["embeddings"]

# Evaluate all 44 models on novel items
novel_results = []
for slug in slugs:
    model_rf = joblib.load(f"atlas/models/{slug}_rf_model.pkl")
    enc = joblib.load(f"atlas/models/{slug}_label_encoder.pkl")
    
    idx = [i for i, item in enumerate(test_items) if item["slug"] == slug]
    if not idx:
        continue
    
    X_novel = test_emb[idx]
    y_true = [test_items[i]["expected_factor"] for i in idx]
    y_pred = enc.inverse_transform(model_rf.predict(X_novel))
    acc = accuracy_score(y_true, y_pred)
    novel_results.append({"Model": slug.upper(), "Items": len(idx), "Novel Accuracy": f"{acc:.1%}", "_acc": acc})

novel_df = pd.DataFrame(novel_results).sort_values("_acc", ascending=False).drop(columns=["_acc"])
novel_df.index = range(1, len(novel_df) + 1)

mean_novel = np.mean([r["_acc"] for r in novel_results])
print(f"Mean novel-item accuracy: {mean_novel:.1%} (vs. ~98% on training data)")
print(f"This is the generalization accuracy reported in the paper.\n")
novel_df

## 8. Experiment 2 — 3072-dim Embedding Upgrade

OpenAI's `text-embedding-3-large` produces 3,072-dim vectors — twice the resolution of the default. Does doubling the embedding dimensionality improve factor classification?

The upgraded embeddings (440 MB) and retrained classifiers (107 MB) are hosted on [Hugging Face Hub](https://huggingface.co/datasets/Wildertrek/personality-atlas-3072). The cells below download them on demand and compare accuracy side by side — first on training data, then on the novel test items.

In [ ]:
!pip install -q huggingface_hub

import os
os.environ["HF_HUB_DISABLE_TELEMETRY"] = "1"

import warnings
warnings.filterwarnings("ignore", message=".*HF_TOKEN.*")
warnings.filterwarnings("ignore", message=".*unauthenticated.*")

from huggingface_hub import hf_hub_download

HF_REPO = "Wildertrek/personality-atlas-3072"

def load_3072(slug):
    """Download and load 3072-dim assets for a single model from HuggingFace."""
    emb_path = hf_hub_download(HF_REPO, f"Embeddings_3072/{slug}_embeddings.csv", repo_type="dataset")
    model_path = hf_hub_download(HF_REPO, f"models_3072/{slug}_rf_model.pkl", repo_type="dataset")
    enc_path = hf_hub_download(HF_REPO, f"models_3072/{slug}_label_encoder.pkl", repo_type="dataset")
    emb_df = pd.read_csv(emb_path)
    X = np.array([ast.literal_eval(e) for e in emb_df["Embedding"]])
    return X, joblib.load(model_path), joblib.load(enc_path)

print("Ready to download 3072-dim assets from HuggingFace.")

In [ ]:
# Compare 1536 vs 3072 accuracy across all 44 models
comparison = []
for slug in slugs:
    df = pd.read_csv(f"atlas/datasets/{slug}.csv")
    y_true = df["Factor"].values

    # 1536-dim (already loaded from repo)
    m1536 = joblib.load(f"atlas/models/{slug}_rf_model.pkl")
    e1536 = joblib.load(f"atlas/models/{slug}_label_encoder.pkl")
    emb1536 = pd.read_csv(f"atlas/Embeddings/{slug}_embeddings.csv")
    X1536 = np.array([ast.literal_eval(e) for e in emb1536["Embedding"]])
    acc1536 = (e1536.inverse_transform(m1536.predict(X1536)) == y_true).mean()

    # 3072-dim (downloaded from HuggingFace)
    X3072, m3072, e3072 = load_3072(slug)
    acc3072 = (e3072.inverse_transform(m3072.predict(X3072)) == y_true).mean()

    delta = (acc3072 - acc1536) * 100
    comparison.append({
        "Model": slug.upper(),
        "1536-dim": f"{acc1536:.1%}",
        "3072-dim": f"{acc3072:.1%}",
        "Delta": f"{delta:+.1f}pp",
        "_delta": delta,  # numeric for sorting
    })

comp_df = pd.DataFrame(comparison).sort_values("_delta", ascending=False).drop(columns=["_delta"])
comp_df.index = range(1, len(comp_df) + 1)

improved = sum(1 for c in comparison if c["_delta"] > 0.05)
decreased = sum(1 for c in comparison if c["_delta"] < -0.05)
unchanged = len(comparison) - improved - decreased
mean_1536 = np.mean([float(c["1536-dim"].rstrip("%")) for c in comparison])
mean_3072 = np.mean([float(c["3072-dim"].rstrip("%")) for c in comparison])

print(f"1536-dim mean: {mean_1536:.1f}% | 3072-dim mean: {mean_3072:.1f}% | Delta: {mean_3072 - mean_1536:+.1f}pp")
print(f"{improved} improved, {decreased} decreased, {unchanged} unchanged\n")
comp_df

### Novel item accuracy: 1536-dim vs 3072-dim

The comparison above used each model's own training data, where both embedding sizes tend to score high. The more informative test: run both classifiers on the **5,052 novel items** from Section 7 — items neither classifier has seen during training. Pre-computed 3072-dim embeddings for these items are downloaded from HuggingFace.

In [ ]:
# Novel item accuracy: 1536-dim vs 3072-dim on truly novel test items
# Downloads 3072-dim test item embeddings from HuggingFace (no API key needed)
test3072_path = hf_hub_download(HF_REPO, "test_items/test_items_embeddings_3072.npz", repo_type="dataset")
test_emb_3072 = np.load(test3072_path)["embeddings"]

novel_comparison = []
for slug in slugs:
    m1536 = joblib.load(f"atlas/models/{slug}_rf_model.pkl")
    e1536 = joblib.load(f"atlas/models/{slug}_label_encoder.pkl")
    X3072_m, m3072, e3072 = load_3072(slug)

    idx = [i for i, item in enumerate(test_items) if item["slug"] == slug]
    if not idx:
        continue

    y_true = [test_items[i]["expected_factor"] for i in idx]
    acc_n1536 = accuracy_score(y_true, e1536.inverse_transform(m1536.predict(test_emb[idx])))
    acc_n3072 = accuracy_score(y_true, e3072.inverse_transform(m3072.predict(test_emb_3072[idx])))
    delta = (acc_n3072 - acc_n1536) * 100
    novel_comparison.append({
        "Model": slug.upper(), "Items": len(idx),
        "1536-dim": f"{acc_n1536:.1%}", "3072-dim": f"{acc_n3072:.1%}",
        "Delta": f"{delta:+.1f}pp", "_delta": delta
    })

nc_df = pd.DataFrame(novel_comparison).sort_values("_delta", ascending=False).drop(columns=["_delta"])
nc_df.index = range(1, len(nc_df) + 1)

mean_n1536 = np.mean([float(c["1536-dim"].rstrip("%")) for c in novel_comparison])
mean_n3072 = np.mean([float(c["3072-dim"].rstrip("%")) for c in novel_comparison])
improved = sum(1 for c in novel_comparison if c["_delta"] > 0.05)
decreased = sum(1 for c in novel_comparison if c["_delta"] < -0.05)
unchanged = len(novel_comparison) - improved - decreased

print(f"Novel item accuracy: 1536-dim mean: {mean_n1536:.1f}% | 3072-dim mean: {mean_n3072:.1f}% | Delta: {mean_n3072 - mean_n1536:+.1f}pp")
print(f"{improved} improved, {decreased} decreased, {unchanged} unchanged\n")
nc_df

## 9. Experiment 3 — DSM-5 Clinical Alignment

The atlas was built from personality research instruments, not clinical diagnostic tools. But personality and psychopathology overlap heavily — the DSM-5 includes dimensional personality models, and clinical instruments like the MMPI and SCID already sit in the atlas.

If the taxonomy is well-structured, clinical constructs from outside the training data should route to the correct category. We test this by embedding all **222 DSM-5-TR disorders** (using each disorder's name and diagnostic keywords) and querying them against the atlas FAISS index. For each disorder, the top-10 nearest traits vote on a category.

The question: what fraction land in Clinical/Health? Data and pre-computed embeddings are included in the repository.

In [ ]:
# Classify 222 DSM-5-TR disorders through the atlas
# Uses pre-computed embeddings — no API key needed
import json
from collections import Counter

dsm5 = json.load(open("atlas/data/dsm5_disorders.json"))
dsm5_emb = pd.read_csv("atlas/data/dsm5_embeddings.csv")
dsm5_vecs = np.array([ast.literal_eval(e) for e in dsm5_emb["Embedding"]]).astype(np.float32)
dsm5_vecs = dsm5_vecs / np.linalg.norm(dsm5_vecs, axis=1, keepdims=True)

# Query each disorder against the atlas FAISS index (built in Section 5)
dsm5_results = []
for i, disorder in enumerate(dsm5):
    q = dsm5_vecs[i:i+1]
    D, I = index.search(q, 10)
    top_cats = [all_cats[j] for j in I[0]]
    predicted_cat = Counter(top_cats).most_common(1)[0][0]
    dsm5_results.append({
        "disorder": disorder["disorder_name"],
        "dsm5_category": disorder["dsm5_category"],
        "predicted_atlas_cat": predicted_cat,
        "is_clinical": predicted_cat == "Clinical/Health",
        "top_cat_counts": dict(Counter(top_cats))
    })

clinical_count = sum(1 for r in dsm5_results if r["is_clinical"])
pct = clinical_count / len(dsm5_results) * 100
print(f"DSM-5 Clinical Alignment: {clinical_count}/{len(dsm5_results)} disorders ({pct:.1f}%) route to Clinical/Health")
print(f"This confirms the atlas taxonomy correctly captures clinical constructs.\n")

# Show the few that don't route to Clinical/Health
non_clinical = [r for r in dsm5_results if not r["is_clinical"]]
if non_clinical:
    print(f"{len(non_clinical)} disorders route elsewhere:")
    for r in non_clinical:
        print(f"  {r['disorder'][:60]:<62} → {r['predicted_atlas_cat']}")
else:
    print("All 222 disorders route to Clinical/Health.")

In [ ]:
# Per DSM-5 category breakdown — which clinical domains route where?
dsm5_cats = sorted(set(r["dsm5_category"] for r in dsm5_results))

print(f"{'DSM-5 Category':<45} {'N':>3}  {'Clinical':>8}  {'Other':>5}  {'Pct':>5}")
print("-" * 72)
for cat in dsm5_cats:
    cat_results = [r for r in dsm5_results if r["dsm5_category"] == cat]
    n = len(cat_results)
    clin = sum(1 for r in cat_results if r["is_clinical"])
    other = n - clin
    cat_pct = clin / n * 100
    marker = "" if cat_pct == 100 else " *"
    print(f"{cat:<45} {n:>3}  {clin:>8}  {other:>5}  {cat_pct:>4.0f}%{marker}")

overall_pct = clinical_count / len(dsm5_results) * 100
print(f"\n* = categories where some disorders route outside Clinical/Health")
print(f"Total: {len(dsm5_results)} disorders, {clinical_count} ({overall_pct:.1f}%) route to Clinical/Health")

---

The full pipeline — from raw psychometric instruments to a searchable, classifiable embedding space — is documented in the paper and fully reproducible from this repository.

**Want to go deeper?** The [Deep Dive notebook](https://colab.research.google.com/github/Wildertrek/survey/blob/main/notebooks/atlas_deep_dive.ipynb) lets you pick any single model and take it apart: train a classifier from scratch, inspect the confusion matrix, visualize embedding geometry, and measure inter-factor similarity.

**Repository:** [github.com/Wildertrek/survey](https://github.com/Wildertrek/survey) | **3072-dim assets:** [Hugging Face Hub](https://huggingface.co/datasets/Wildertrek/personality-atlas-3072)
**Paper:** Raetano, J., Gregor, J., & Tamang, S. (2026). *A Survey and Computational Atlas of Personality Models.* ACM TIST.
**License:** MIT